In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:37676")
client

Client Scheduler: tcp://127.0.0.1:37676 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 9 Cores: 72 Memory: 810.00 GB


In [3]:
import dask.dataframe as dd
from pathlib import Path
import glob

for csv_file in glob.glob("../../../data/train/viznet/ngram/*.csv"):
    file_path = Path(csv_file)
    
    df = dd.read_csv(file_path, keep_default_na=False, dtype=str)
    df = df.rename(columns={"Unnamed: 0": "data", "2_gram": "count", "3_gram": "count", "4_gram": "count"})
    print(df.columns)
    output_path = file_path.parent / file_path.stem
    output_path.mkdir(parents=True, exist_ok=True)
    df.to_json(str(output_path), compute=True, compression="gzip")

Index(['data', 'count'], dtype='object')
Index(['data', 'count'], dtype='object')
Index(['data', 'count'], dtype='object')
Index(['data', 'count'], dtype='object')
Index(['data', 'count'], dtype='object')
Index(['data', 'count'], dtype='object')
Index(['data', 'count'], dtype='object')
Index(['data', 'count'], dtype='object')
Index(['data', 'count'], dtype='object')
Index(['data', 'count'], dtype='object')
Index(['data', 'count'], dtype='object')


In [4]:
from elasticsearch import Elasticsearch, helpers

es = Elasticsearch()

In [5]:
char_ngram_indexed = True
tok_ngram_indexed = True
n_indices_indexed = True
indices_indexed = True

if not es.indices.exists(index="char_ngram"):
    es.indices.create(
        index="char_ngram",
        body={
            "settings": {"number_of_shards": 5},
            "mappings": {
                "properties": {"data": {"type": "keyword"}, "count": {"type": "long"},}
            },
        },
        ignore=400,
    )
    char_ngram_indexed = False

if not es.indices.exists(index="tok_ngram"):
    es.indices.create(
        index="tok_ngram",
        body={
            "settings": {"number_of_shards": 5},
            "mappings": {
                "properties": {"data": {"type": "keyword"}, "count": {"type": "long"},}
            },
        },
        ignore=400,
    )
    tok_ngram_indexed = False

if not es.indices.exists(index="reversed_indices"):
    es.indices.create(
        index="reversed_indices",
        body={
            "settings": {"number_of_shards": 5},
            "mappings": {
                "properties": {"data": {"type": "keyword"}, "idx": {"type": "long"},}
            },
        },
        ignore=400,
    )
    indices_indexed = False

if not es.indices.exists(index="n_reversed_indices"):
    es.indices.create(
        index="n_reversed_indices",
        body={
            "settings": {"number_of_shards": 5},
            "mappings": {
                "properties": {"data": {"type": "keyword"}, "idx": {"type": "long"},}
            },
        },
        ignore=400,
    )
    n_indices_indexed = False

In [6]:
import gzip
from collections import deque
from pathlib import Path

import rapidjson as json

for sub_path in Path("../../../data/train/viznet/ngram").iterdir():
    if "2_gram_count" not in sub_path.name:
        continue
    if sub_path.is_dir() and "token" not in sub_path.name and not char_ngram_indexed:
        for file_path in sub_path.iterdir():
            print(file_path)
            if file_path.suffix == ".part":
                with gzip.GzipFile(file_path, "r") as fin:
                    data = [json.loads(l) for l in fin.readlines()]
                    print(data[:1])
                    try:
                        deque(
                            helpers.parallel_bulk(es, data, index="char_ngram"),
                            maxlen=0,
                        )
                    except Exception as e:
                        print(i, e)
                        continue
    elif sub_path.is_dir() and not tok_ngram_indexed:
        for file_path in sub_path.iterdir():
            print(file_path)
            if file_path.suffix == ".part":
                with gzip.GzipFile(file_path, "r") as fin:
                    data = [json.loads(l) for l in fin.readlines()]
                    try:
                        deque(
                            helpers.parallel_bulk(es, data, index="tok_ngram"), maxlen=0
                        )
                    except Exception as e:
                        print(e)
                        continue


# for sub_path in Path("../../../data/train/viznet").iterdir():
#     print(sub_path)
#     if sub_path.suffix == ".jsonl":
#         if "pattern_n_cooc" in str(sub_path) and not n_indices_indexed:
#             print("Indexing")
#             with open(sub_path, "r") as fin:
#                 data = [json.loads(l) for l in fin.readlines()]
#                 for i in range(0, len(data), 50):
#                     try:
#                         deque(
#                             helpers.parallel_bulk(
#                                 es, data[i : i + 50], index="n_reversed_indices"
#                             ),
#                             maxlen=0,
#                         )
#                     except Exception as e:
#                         print(i, e)
#                         continue
#         elif not indices_indexed:
#             with open(sub_path, "r") as fin:
#                 data = [json.loads(l) for l in fin.readlines()]
#                 for i in range(0, len(data), 1000):
#                     try:
#                         deque(
#                             helpers.parallel_bulk(
#                                 es, data[i : i + 50], index="reversed_indices"
#                             ),
#                             maxlen=0,
#                         )
#                     except Exception as e:
#                         print(i, e)
#                         continue